In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# import sys
# sys.path.append('..')  # Add the parent directory to the sys.path
from model.model_fast import City

In [ ]:
# Run model
num_steps  = 10
parameters = {
            'width':     3,
            'height':    3,
}
city = City(num_steps, **parameters)
for time_step in range(num_steps): # TODO time here and above
    # print(f'Step {time}')
    city.step_fast()
city.record_run_data_to_file()
agent_out = city.datacollector.get_agent_vars_dataframe()
model_out = city.datacollector.get_model_vars_dataframe()

In [ ]:
# Plot 
# Assuming model_out['R_N'], model_out['investor_bid'], etc., are pandas Series
attributes_to_plot = ["R_N", "investor_bid", "warranted_rent", "warranted_price", "maintenance"]

# Create subplots
num_rows = len(attributes_to_plot)
num_cols = 1  # You can adjust the number of columns as needed
fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 6 * num_rows))



# first_elements = [lst[0] for lst in model_out['R_N']]

# Loop over attributes
for i, attribute in enumerate(attributes_to_plot):
    # Get the values for the current attribute
    values_by_position = np.array([lst for lst in model_out[attribute]])

    # Plot each value as a point against time_step
    for position in range(values_by_position.shape[1]):
        values = values_by_position[:, position]
        axs[i].scatter(model_out['time_step'], values, label=f"Position {position + 1}")

        # Connect points with lines
        axs[i].plot(model_out['time_step'], values, linestyle='--')

    axs[i].set_xlabel('Time Step')
    axs[i].set_ylabel(f'Values from {attribute}')
    axs[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming model_out['R_N'] is a pandas Series
values_by_position = np.array([lst for lst in model_out['R_N']])

# Plotting
num_positions = values_by_position.shape[1]

# Get distinct colors for each position
colors = plt.cm.viridis(np.linspace(0, 1, num_positions))

# Create subplots
fig, ax = plt.subplots(figsize=(10, 6))

# Plot each value as a point against time_step
for position in range(num_positions):
    values = values_by_position[:, position]
    ax.scatter(model_out['time_step'], values, label=f"Position {position + 1}", color=colors[position])

    # Connect points with lines
    ax.plot(model_out['time_step'], values, color=colors[position], linestyle='--')

ax.set_xlabel('Time Step')
ax.set_ylabel('Values from R_N')
ax.legend()
plt.show()


In [ ]:
# Extract attributes for plotting (excluding 'dist')
attributes_to_plot = ["R_N", "investor_bid", "warranted_rent", "warranted_price", "maintenance"]

# Create subplots
num_rows = 4
num_cols = 2
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
fig.suptitle("Attributes vs Distance at Each Time Step", y=1.02, fontsize='x-large')  # Increased font size for the title

# Get distinct colors for each time step
num_time_steps = city.num_steps
colors = plt.cm.viridis(np.linspace(0, 1, num_time_steps))

# Loop over attributes
for i, attribute in enumerate(attributes_to_plot):
    # Loop over time steps
    for time_step in range(2, num_time_steps + 1):
        # Load data for the current time step
        loaded_data = model_out[model_out['time_step'] == time_step]

        # Extract all values from the list for each time step
        investor_bid_values = loaded_data[attribute].apply(
            lambda x: x if isinstance(x, list) and len(x) > 0 else [x]
        )

        # Plot all values on the same plot
        for j, bids in enumerate(investor_bid_values.tolist()):
            axs[i // num_cols, i % num_cols].plot(
                [time_step] * len(bids),
                bids,
                marker='o',
                label=f"{attribute} - Time Step {time_step}",
                color=colors[time_step - 1]
            )

    axs[i // num_cols, i % num_cols].set_title(attribute, fontsize='large')  # Increased font size for the title
    axs[i // num_cols, i % num_cols].set_xlabel("Time Step", fontsize='medium')  # Increased font size for x-axis label
    axs[i // num_cols, i % num_cols].set_ylabel(attribute, fontsize='medium')  # Increased font size for y-axis label

# Add a legend to the last subplot
legend_ax = axs[-1, -1]
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, title='Time Steps', bbox_to_anchor=(.9, .10), loc='lower right', fontsize='large')
legend_ax.axis('off')  # Disable axis for the legend

plt.tight_layout()
plt.show()


In [ ]:
# Extract attributes for plotting
attributes_to_plot = ["city_extent_calc"]

# Create subplots
num_rows = 1
num_cols = 1
fig, axs = plt.subplots(num_rows, num_cols, figsize=(8, 6))
fig.suptitle("City Extent Calculation Over Time", y=1.02, fontsize='x-large')  # Increased font size for the title

# Load data
loaded_data = model_out

# Plot each attribute
for attribute in attributes_to_plot:
    axs.plot(
        loaded_data['time_step'],
        loaded_data[attribute],
        label=attribute
    )
    axs.set_xlabel("Time Step", fontsize='medium')  # Increased font size for x-axis label
    axs.set_ylabel("City Extent Calculation", fontsize='medium')  # Increased font size for y-axis label

# Add a legend
axs.legend(title='Attributes', loc='upper left', fontsize='large')

plt.tight_layout()
plt.show()

In [ ]:
# Plot 
for time_step in range(1, num_steps + 1):
    # Load newcomer bid data for the current time step
    loaded_data = model_out[model_out['time_step'] == time_step]['newcomer_bid'].values[0]

    # Extract data
    newcomer_bids, dist_values, savings_values = zip(*loaded_data)

    # Create a 3D surface plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(dist_values, savings_values, newcomer_bids, cmap='viridis', edgecolor='k')
    ax.scatter(dist_values, savings_values, newcomer_bids, marker='*', color='red', s=100, label='Newcomer Bids')
    ax.set_title(f'Newcomer Bids - Time Step {time_step}')
    ax.set_xlabel('Distance')
    ax.set_ylabel('Savings')
    ax.set_zlabel('Newcomer Bid')
    plt.show()

In [ ]:
print(model_out['R_N'][0])